# Scrapping de données météo

Dans ce notebook, on cherche à récupérer les données météo de plusieurs communes, afin de rendre pertinente une régression de la consommation d'énergie en fonction de l'indice de performance du bâtiment et de la météo.

Cette analyse s'inscrit dans une dimension bien plus locale que l'analyse précédente, qui portait sur une échelle départementale et non communale.

In [62]:
!pip install pandas
!pip install geopandas
!pip install geoviews
!pip install lxml
!pip install urllib
!pip install matplotlib
!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a
!pip install mapclassify
!pip install folium

  Using cached geoviews-1.11.0-py2.py3-none-any.whl.metadata (11 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 89.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.2/511.2 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 92.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 56.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 78.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 20.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement urllib (from versions: none)
ERROR: No matc

In [1]:
!pip install -q lxml

In [64]:
from declarations import *

ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=DEPARTEMENT/crs=4326/FRANCE_ENTIERE=metropole/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 7.17MiB [00:00, 97.4MiB/s]


In [16]:
import bs4
import lxml
import pandas as pd
import urllib
from tqdm.auto import tqdm

from urllib import request

In [3]:
""" Création du dictionnaire Racine """

data_all = {
    i : {j : {} for j in range(1,13)} for i in range(2011,2019)
}

In [4]:
""" Scrapping du site infoclimat.fr et organisation des données sous forme de dictionnaire """
# Il faut optimiser la création en ne créant qu'une seule fois le Df lié aux villes

for i in tqdm(range(len(data_all))):
    for y in data_all :
        for m in data_all[y] :
            url_test = 'https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=' + str(m) + '&annee=' + str(y)
            raw_text = request.urlopen(url_test).read()
            page = bs4.BeautifulSoup(raw_text) #,'lxml')
            tableau = page.find('table', {'id' : 'tableau-releves'})
            rows = tableau.find_all('tr')
    
            df = {
        'Villes' : [rows[i].find('a').text for i in range(1,len(rows))], #[rows[i].find_all('div')[0].text for i in range(1,len(rows))], #[rows[i].find('a').text for i in range(1,len(rows))],
        'Température minimale extrême du mois' : [rows[i].find('div').text for i in range(1,len(rows))],
        'Moyenne des températures minimales du mois' : [rows[i].find_all('td')[2].text for i in range(1,len(rows))],
        'Température moyenne du mois' : [rows[i].find_all('td')[3].text for i in range(1,len(rows))],
        'Moyenne des températures maximales du mois' : [rows[i].find_all('td')[4].text for i in range(1,len(rows))],
        'Température maximale extrême du mois' : [rows[i].find_all('td')[5].find('div').text for i in range(1,len(rows))],
        'Cumul de précipitation du mois' : [rows[i].find_all('td')[6].text for i in range(1,len(rows))],
        "Heure d'ensolleiment du mois" : [rows[i].find_all('td')[7].text for i in range(1,len(rows))],
        'Rafale maximale du mois' : [rows[i].find_all('td')[8].text for i in range(1,len(rows))] 
                }
    
            data_all[y][m] = df

  0%|          | 0/8 [00:00<?, ?it/s]

## Création d'une liste de ville

On garde pour la suite le nom des stations utiles. Ces noms nous permettront de filtrer les communes dont nous devons étudier les DPE.

In [ ]:
df['Villes']

On voit que les noms de stations ne sont pas très propres : ils incluent les numéros de département, voire incluent des quartiers ou regroupent plusieurs villes. On cherche donc à nettoyer ces noms, pour qu'ils puissent être comparés aux noms de commune donnée par l'API utile pour les DPE. 

On ne touche pour cela qu'à la liste "Villes" du data frame. Nous remplacerons ensuite la base initiale par la base nettoyée, et supprimons les lignes qui ne nous intéressent pas dans le data frame après nettoyage. 

L'API des DPE qui nous intéresse n'identifie une commune que par son code INSEE. Il faut donc que, dans la base de données de chaque station, je réussisse à merger la commune de la station avec une base de données qui comprend les codes communes INSEE. 

In [28]:
#Création d'un Data Frame à partir de la colonne "Villes" du dictionnaire précédent
stations=pd.DataFrame(df['Villes'],columns=['Ville'])
stations

,Ville
0,Abbeville (80)
1,Agde (34)
2,Agde - Le Grau (34)
3,Agen - La Garenne (47)
4,Aigrefeuille-d'Aunis (17)
...,...
873,[MAE] Lycée Roz Glas - QUIMPERLE (29)
874,[MAE] Lycée Saint Exupéry - LA ROCHELLE (17)
875,[MAE] Lycée Victor Louis - TALENCE (33)
876,[MAE] Lycée Victor Louis - TALENCE (33)


In [54]:
#On ne garde que les stations dans les villes et pas celles de stations de "Météo à l'école" (MAE)
liste_communes=stations[~stations['Ville'].str[0].isin(['['])]
liste_communes

,Ville
0,Abbeville (80)
1,Agde (34)
2,Agde - Le Grau (34)
3,Agen - La Garenne (47)
4,Aigrefeuille-d'Aunis (17)
...,...
781,Xonrupt-Longemer (88)
782,Xonrupt-Longemer - Le Poli (88)
783,Xonrupt-Longemer - Refuge du Sotré (88)
784,Yssingeaux (43)


On voit qu'en plus par ville il peut il y avoir plusieurs stations (correspondant à plusieurs quartiers). On va donc essayer de supprimer les parties qui ne dénotent pas de la ville dans les noms de stations.

In [ ]:
# Méthode très naïve
def nettoyage(liste):
    '''Cette fonction est censée séparer les numéros de département des villes dans une liste présentée comme la liste liste_communes
    Argument : liste (type : list ; liste de string) 
    Sortie : List (type : list ; liste de listes)'''
    List=[]
    for ville in liste:
        i=0
        mot=''
        while i < len(ville) and ville[i+1] != '(' :
            mot+=ville[i]
            i+=1
        List.append([mot,ville[i+2:-1]])  
    return List
List=nettoyage(liste_communes['Ville'])
List

In [52]:
""" Conversion en DataFrame pour exporter """

df_all = pandas.DataFrame(data_all)
df_all

,2011,2012,2013,2014,2015,2016,2017,2018
1,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
2,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
3,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
4,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
5,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
6,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
7,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
8,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
9,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag..."
10,"{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (34)', 'Ag...","{'Villes': ['Abbeville (80)', 'Agde (

In [6]:
""" Exportation au format json """

df_all.to_json('data_base.json')

In [9]:
""" Normalisation des variables suivant la nomenclature de la source """

df_all.rename({'Villes' : 'ville', 
           'Température minimale extrême du mois' : 'tnn',
           'Moyenne des températures minimales du mois' : 'tnm', 
           'Température moyenne du mois' : 'tmm', 
           'Moyenne des températures maximales du mois' : 'txm', 
           'Température maximale extrême du mois' : 'txx', 
           'Cumul de précipitation du mois' : 'rr', 
           "Heure d'ensolleiment du mois" : 'ens', 
           'Rafale maximale du mois' : 'rafale'},axis=1,inplace = True)

In [10]:
class Meteo :
    def __init__(self):
        self.data_all = {
                i : {j : {} for j in range(1,13)} for i in range(2011,2019)
            }

    def scrap(self):
        for y in self.data_all :
            for m in self.data_all[y] :
                url_test = 'https://www.infoclimat.fr/stations-meteo/analyses-mensuelles.php?mois=' + str(m) + '&annee=' + str(y)
                raw_text = request.urlopen(url_test).read()
                page = bs4.BeautifulSoup(raw_text,'lxml')
                tableau = page.find('table', {'id' : 'tableau-releves'})
                rows = tableau.find_all('tr')
        
                df = {
            'Villes' : [rows[i].find('a').text for i in range(1,len(rows))], #[rows[i].find_all('div')[0].text for i in range(1,len(rows))], #[rows[i].find('a').text for i in range(1,len(rows))],
            'Température minimale extrême du mois' : [rows[i].find('div').text for i in range(1,len(rows))],
            'Moyenne des températures minimales du mois' : [rows[i].find_all('td')[2].text for i in range(1,len(rows))],
            'Température moyenne du mois' : [rows[i].find_all('td')[3].text for i in range(1,len(rows))],
            'Moyenne des températures maximales du mois' : [rows[i].find_all('td')[4].text for i in range(1,len(rows))],
            'Température maximale extrême du mois' : [rows[i].find_all('td')[5].find('div').text for i in range(1,len(rows))],
            'Cumul de précipitation du mois' : [rows[i].find_all('td')[6].text for i in range(1,len(rows))],
            "Heure d'ensolleiment du mois" : [rows[i].find_all('td')[7].text for i in range(1,len(rows))],
            'Rafale maximale du mois' : [rows[i].find_all('td')[8].text for i in range(1,len(rows))] 
                    }
        
                self.data_all[y][m] = df

    def export(self):
        self.df_all = pandas.DataFrame(self.data_all)
        self.df_all.to_json('data_base.json')

    def rename(self):
        self.df_all = pandas.DataFrame(self.data_all)
        self.df_all.rename({'Villes' : 'ville', 
           'Température minimale extrême du mois' : 'tnn',
           'Moyenne des températures minimales du mois' : 'tnm', 
           'Température moyenne du mois' : 'tmm', 
           'Moyenne des températures maximales du mois' : 'txm', 
           'Température maximale extrême du mois' : 'txx', 
           'Cumul de précipitation du mois' : 'rr', 
           "Heure d'ensolleiment du mois" : 'ens', 
           'Rafale maximale du mois' : 'rafale'},axis=1,inplace = True)
    

In [66]:
dep = s3.download_vectorfile_url_all(
    values = "metropole",
    crs = 4326,
    borders = "DEPARTEMENT",
    vectorfile_format="topojson",
    filter_by="FRANCE_ENTIERE",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

dep["area"] = dep.to_crs(2154).area

https://minio.lab.sspcloud.fr/projet-cartiflette/diffusion/shapefiles-test1/year=2022/administrative_level=DEPARTEMENT/crs=4326/FRANCE_ENTIERE=metropole/vectorfile_format=topojson/provider=IGN/source=EXPRESS-COG-CARTO-TERRITOIRE/raw.topojson


Downloading: : 7.17MiB [00:00, 92.0MiB/s]


### Récupération des données de Diagnostics de Performance Energétique (DPE)

On commence par programmer une fonction qui permet de récupérer des données à partir d'une APL.

In [ ]:
def get_dpe_from_url(api_url):
    '''Argument:
        api_url (string) : url de l'API voulue
    Sortie:
        pandas.DataFrame : dataframe
    '''
    req=requests.get(api_url)
    wb=req.json()
    df = pd.json_normalize(wb["results"])
    dpe = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs = 4326)
    dpe = dpe.dropna(subset = ['longitude', 'latitude'])
    return dpe

Il est trop coûteux de récupérer l'intégralité des données de DPE, et nous ne disposons de données météo que pour certaines communes. L'idée est donc de ne récupérer les données de DPE que pour les communes voulues et de les agréger dans une unique table de données.

In [ ]:
def get_dpe_dataset(api_root, communes):
    '''
    Args : 
    Returns : 
    '''
    